<div align="center">
  <h1><b> Quantum Computing </b></h1>
  <h2> Variational Quantum Eigensolver (VQE) </h2>
</div>

<br>
<b>Author:</b> <a target="_blank" href="https://github.com/camponogaraviera">Lucas Camponogara Viera</a>

# Table of Contents

- [Introduction](#introduction)

- [Background](#background)
  - The Variational Principle
  - Advantages
  - Requirements
  - The VQE Algorithm
  
- [Qiskit Implementation](#qiskit-implementation)
  - Imports
  - Information About Molecules
  - Qubit Operator Function 
  - Exact Solution Function
  - Ansatz
  - Adapt VQE Function
  - LiH Simulation

# Introduction

The **Variational Quantum Eigensolver (VQE)** \[[1](#1)]\[[2](#2)] is a flagship among near-term quantum algorithms. It is a **hybrid quantum-classical algorithm** designed to estimate the **ground-state energy** of a given **Hamiltonian**, a fundamental problem in quantum chemistry and materials science. By leveraging both quantum and classical computational resources, VQE provides a practical approach for solving problems that are intractable for purely classical algorithms.

VQE is an application of the **Ritz variational principle** used to find the ground state corresponding to the lowest energy level ($E_o$) of a chemical molecule. The ground state of a molecule provides access to information of **molecular reaction rates** ($r \propto e^{\mathcal{O}(\Delta E_0)}$) and molecular structure used to drive the design of new drugs, materials, and electric batteries.

# Background


In quantum mechanics, the state of a system is described by a *wavefunction* $|\psi\rangle$, and its energy is determined by the **Hamiltonian operator** $\hat{H}$. The ground-state energy $E_0$ is given by:

$$
E_0 = \min_{|\psi\rangle} \langle \psi | \hat{H} | \psi \rangle
$$

Finding this minimum directly is computationally expensive since the Hilbert space grows exponentially with system size.

## The Variational Principle

The **variational principle** states that for any trial wavefunction $|\psi(\boldsymbol{\theta})\rangle$, parameterized by a set of real parameters $\boldsymbol{\theta}$, the expectation value of the Hamiltonian provides an upper bound to the ground-state energy:

$$
E(\boldsymbol{\theta}) = \langle \psi(\boldsymbol{\theta}) | \hat{H} | \psi(\boldsymbol{\theta}) \rangle \geq E_0
$$

Thus, by optimizing $\boldsymbol{\theta}$ to minimize $E(\boldsymbol{\theta})$, one can approximate the ground state and its energy.

## Advantages

1. **Noise resilience:** Suitable for near-term **Noisy Intermediate-Scale Quantum (NISQ)** devices.  

2. **Flexibility:** Adaptable to various Hamiltonians and problem domains.  

3. **Scalability:** Reduces the quantum circuit depth by offloading part of the computation to classical optimization.

## Requirements

1. **Coherence friendly:** the circuit must be shallow, i.e, have a small number of layers (small circuit depth) in order to be executed during a time window shorter than decoherence time.
   
2. **Hardware friendly (qubit routing):** gate coupling must be allowed only between nearest-neighbor physical qubits etched into the hardware processor in order to avoid non-trivial transpilation, i.e, the use of SWAP gates (non-native) during qubit routing (mapping from the circuit diagram to a hardware topology).

3. **Small number of hyperparameters:** the algorithm must seek the minimum number of angles to be optimized in order to avoid classical optimization overhead (when classical computation becomes too expensive).

## The VQE Algorithm

VQE operates in a **hybrid loop**:
1. A **quantum computer** prepares the trial state $|\psi(\boldsymbol{\theta})\rangle$ using a **parameterized quantum circuit** (ansatz). After various measurements, it is possible to compute the expectation value of the Hamiltonian.  

2. A **classical optimizer** updates the parameters $\boldsymbol{\theta}$ via the **parameter shift rule** to minimize the energy $E(\boldsymbol{\theta})$.  

This iterative process continues until convergence, yielding an estimate of the ground-state energy.

Algorithm Outline

1. With the spin orbitals of the molecule of interest, compute its second-quantized electronic Hamiltonian in the STO-3G basis.

2. Map the fermionic Hamiltonian given in terms of its fermionic operators into a spin qubit-equivalent Hamiltonian using either Bravyi-Kitaev or the inverted Jordan–Wigner transformation. This is required to perform gate-based quantum computation. The Hamiltonian will be used to compute the cost function that evaluates the expectation value of the Hamiltonian, while the number of qubits is required to obtain the electronic excitations and to set up the quantum circuit.

3. Obtain the single and double electronic excitations by acting with the electron annihilation and creation operators on the Hartree-Fock reference state.

4. Define a set with all unique single and double excitations to create the correspondent SO(2) unitary qubit gates (the particular Givens rotations) to each electronic excitation operator in order to build a quantum circuit ansatz of particle-conserving unitary qubit gates.

5. Initialize the qubit state to a reference Hartree-Fock state.

6. Initialize the parameter values of each gate in the ansatz to zero, i.e, initialize the ansatz to the identity matrix in order to compute the gradients with respect to the Hartree-Fock state.

7. Prepare the trial estate with the current ansatz.

8. Define the cost function as the expectation value of the qubit Hamiltonian. Define an optimizer (e.g. SGD).

9. Use the parameter shift rule to compute the gradient of the cost function with respect to its tunable parameters.

10. Use the optimizer to update the circuit parameters according to a VQE experiment.

11. Define convergence as the difference between the ground truth and the expected value for the current optimization step. If the convergence tolerance is less than or equal to a pre-defined threshold $\epsilon$, exit the optimization loop and evaluate the cost metric of the final optimized circuit by measuring its circuit depth.

12. Repeat step 9.



# Qiskit Implementation

The following implementation works for Qiskit version 1.4.

Migration to Qiskit version 2.2.3 is ongoing.

## Colab GPUs

In [41]:
!nvidia-smi

Wed Nov 26 14:22:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060        Off |   00000000:07:00.0  On |                  N/A |
|  0%   41C    P8             22W /  190W |     412MiB /   6144MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## &nbsp; <img valign="middle" height="45px" src="https://img.icons8.com/python" width="45" hspace="0px" vspace="0px"> Importing modules

In [20]:
pip show qiskit qiskit-aer qiskit-ibm-runtime qiskit-nature qiskit-algorithms

Name: qiskit
Version: 2.2.3
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /home/ibmq/anaconda3/envs/qc-qiskit/lib/python3.11/site-packages
Requires: dill, numpy, rustworkx, scipy, stevedore, typing-extensions
Required-by: qiskit-aer, qiskit-algorithms, qiskit-ibm-runtime, qiskit-nature
---
Name: qiskit-aer
Version: 0.17.2
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /home/ibmq/anaconda3/envs/qc-qiskit/lib/python3.11/site-packages
Requires: numpy, psutil, python-dateutil, qiskit, scipy
Required-by: 
---
Name: qiskit-ibm-runtime
Version: 0.43.1
Summary: IBM Quantum client for Qiskit Runtime.
Home-page: 
Author: 
Auth

## Information About Molecules

### Helper Functions

In [21]:
'''
from pennylane.qchem import Molecule

def get_mol_info_penny(atoms, coordinates):
    mol = Molecule(atoms, coordinates)

    print(
        f"Electrons: {mol.n_electrons}, "
        f"Orbitals: {mol.n_orbitals}, "
        f"Multiplicity: {mol.mult}, "
        f"Charge: {mol.charge}, "
        f"Basis: {mol.basis_name}"
        )
'''

'\nfrom pennylane.qchem import Molecule\n\ndef get_mol_info_penny(atoms, coordinates):\n    mol = Molecule(atoms, coordinates)\n\n    print(\n        f"Electrons: {mol.n_electrons}, "\n        f"Orbitals: {mol.n_orbitals}, "\n        f"Multiplicity: {mol.mult}, "\n        f"Charge: {mol.charge}, "\n        f"Basis: {mol.basis_name}"\n        )\n'

In [22]:
from openfermion import MolecularData
from openfermionpyscf import run_pyscf

def get_mol_info_open_fermion(atoms, coordinates, basis, multiplicity, charge):
    geometry = list(zip(atoms, coordinates)) # [('H', [0.0, 0.0, 0.0]), ('H', [0.0, 0.0, 0.74])]
    basis = 'sto-3g'
    multiplicity = 1
    charge = 0
    mol = MolecularData(geometry, basis, multiplicity, charge, description='H2')
    mol = run_pyscf(mol, run_fci = True, run_ccsd = True)

    print(
        f"Electrons: {mol.n_electrons}, "
        f"Orbitals: {mol.n_orbitals}, "
        f"Multiplicity: {mol.multiplicity}, "
        f"Charge: {mol.charge}, "
        f"Basis: {mol.basis}, "
        f"Qubits: {mol.n_qubits}, "
        f"FCI: {mol.fci_energy}"
    )

### H2

In [23]:
ang = 0.529177 # Bohr to Angstrom.
dist = 0.6614*ang # Interatomic distance in angstrom.
atoms = ["H", "H"]
coordinates = ([0.0, 0.0, -dist], [0.0, 0.0, dist])

- Using OpenFermion:

In [24]:
get_mol_info_open_fermion(atoms, coordinates, basis="sto-3g", multiplicity=1, charge=0)

Electrons: 2, Orbitals: 2, Multiplicity: 1, Charge: 0, Basis: sto-3g, Qubits: 4, FCI: -1.1361891438445317


### H4

In [25]:
ang = 0.529177 # Bohr to Angstrom.
dist = 0.6614*ang # Interatomic distance in angstrom.
atoms = ["H", "H"]
coordinates = ([0.0, 0.0, -dist], [0.0, 0.0, dist])

- Using OpenFermion:

In [26]:
get_mol_info_open_fermion(atoms, coordinates, basis="sto-3g", multiplicity=1, charge=0)

Electrons: 2, Orbitals: 2, Multiplicity: 1, Charge: 0, Basis: sto-3g, Qubits: 4, FCI: -1.1361891438445317


### H6 

In [27]:
ang = 0.529177 # Bohr to Angstrom.
dist = 1.5*ang # Interatomic distance in angstrom.
atoms = ["H", "H", "H", "H", "H", "H"]
coordinates = ([0.0, 0.0, 0.0], [0.0, 0.0, dist], [0.0, 0.0, 2*dist], [0.0, 0.0, 3*dist], [0.0, 0.0, 4*dist], [0.0, 0.0, 5*dist])

- Using OpenFermion:

In [28]:
get_mol_info_open_fermion(atoms, coordinates, basis="sto-3g", multiplicity=1, charge=0)

Electrons: 6, Orbitals: 6, Multiplicity: 1, Charge: 0, Basis: sto-3g, Qubits: 12, FCI: -3.199565894860906


### LiH

In [29]:
ang = 0.529177 # Bohr to Angstrom.
dist = 2.969280527*ang # Interatomic distance in angstrom.
atoms = ["Li", "H"]
coordinates = ([0.0, 0.0, 0.0], [0.0, 0.0, dist])

- Using OpenFermion:

In [30]:
get_mol_info_open_fermion(atoms, coordinates, basis="sto-3g", multiplicity=1, charge=0)

Electrons: 4, Orbitals: 6, Multiplicity: 1, Charge: 0, Basis: sto-3g, Qubits: 12, FCI: -7.882669746647237


### PCA

In [31]:
atoms = ["O", "O", "O", "O", "C", "C", "C", "C", "C", "C", "C", "H", "H", "H", "H", "H", "H"]
coordinates = [[27.341122521, -25.622605707, 0.00000],
            [33.011245349, -28.896367009, 0.00000],
            [35.846212277, -14.165008065, 0.00000],
            [30.176467394, -14.165008065, 0.00000],
            [33.011245349, -19.075461047, 0.00000],
            [30.176467394, -20.712152725, 0.00000],
            [30.176467394, -23.985914028, 0.00000],
            [35.846212277, -20.712152725, 0.00000],
            [33.011245349, -25.622605707, 0.00000],
            [35.846212277, -23.985914028, 0.00000],
            [33.011245349, -15.801699744, 0.00000],
            [28.418455307, -19.697369869, 0.00000],
            [37.604224365, -19.697369869, 0.00000],
            [37.604224365, -25.000696884, 0.00000],
            [25.583488379, -24.607822851, 0.00000],
            [34.768879491, -29.911149865, 0.00000],
            [35.846212277, -12.135253381, 0.00000]]

In [32]:
#get_mol_info_penny(atoms, coordinates)

In [33]:
#get_mol_info_open_fermion(atoms, coordinates, basis="sto-3g", multiplicity=1, charge=0)

## Qubit Operator Function

In [34]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper, ParityMapper
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

def get_qubit_op (
        atoms,
        coordinates,
        multiplicity,
        charge,
        basis,
        num_electrons_to_use,
        num_spatial_orbitals_to_use,
        mapper_name
        ):

    z2symmetry_reduction = None

    # Mol. info with coordinates in Angstrom:
    molecule_info = MoleculeInfo(
        symbols=atoms,
        coords=coordinates,
        multiplicity=multiplicity,
        charge=charge,
    )

    # Driver:
    driver = PySCFDriver.from_molecule(molecule_info, basis=basis)

    # Get the electronic structure problem:
    problem = driver.run()

    print('Num of alpha+beta spin electrons before reduction:', problem.num_alpha + problem.num_beta)
    print('Num of spatial orbitals before reduction:', problem.num_spatial_orbitals)

    '''
    # Reduce the problem size using INACTIVE space approximation:
    transformer = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    )
    '''

    # Reduce the problem size using ACTIVE space approximation:
    transformer = ActiveSpaceTransformer(
        num_electrons=num_electrons_to_use,
        num_spatial_orbitals=num_spatial_orbitals_to_use,
        active_orbitals=None
    )

    # Reduced electronic structure problem:
    problem  = transformer.transform(problem)

    print('\nNum of alpha+beta spin electrons after reduction:', problem.num_alpha + problem.num_beta)
    print('Num of spatial orbitals after reduction:', problem.num_spatial_orbitals)

    # Properties:
    num_particles = problem.num_particles

    # Mapper:
    match mapper_name:
        case "Jordan-Wigner":
            mapper = JordanWignerMapper()
        case "Bravyi-Kitaev":
            mapper = BravyiKitaevMapper()
        case "Parity":
            mapper = ParityMapper(num_particles=num_particles)

    # Get the Operator in the 2nd quantization formalism:
    second_q_ops = problem.second_q_ops()

    # Get the Hamiltonian:
    hamiltonian = second_q_ops[0]

    # Get the qubit operator:
    qubit_op = mapper.map(hamiltonian)

    print("\nNum of qubits:", qubit_op.num_qubits)

    # Apply symmetry:
    if z2symmetry_reduction != None:
        tapered_mapper = problem.get_tapered_mapper(mapper)
        qubit_op = tapered_mapper.map(hamiltonian)

    return problem, hamiltonian, qubit_op, mapper

## Exact Solution Function

In [35]:
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result.total_energies[0].real

## Ansatz

In [36]:
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

def Ansatz(problem, mapper):
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    return ansatz

## Adapt VQE Function

In [37]:
import numpy as np
from qiskit_algorithms import VQE, AdaptVQE
from qiskit.primitives import Estimator, StatevectorEstimator
#from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

def Adapt_VQE(ansatz, operator, optimizer, gradient_threshold, eigenvalue_threshold, max_iterations, mapper=None, problem=None):
    vqe = VQE(Estimator(), ansatz, optimizer())
    vqe.initial_point = np.zeros(ansatz.num_parameters)

    adapt_vqe = AdaptVQE(vqe,
                         gradient_threshold=gradient_threshold,
                         eigenvalue_threshold=eigenvalue_threshold,
                         max_iterations=max_iterations)

    adapt_vqe.supports_aux_operators = lambda: True
    #min_value_result = adapt_vqe.solver.compute_minimum_eigenvalue(operator)

    solver = GroundStateEigensolver(mapper, adapt_vqe)
    result = solver.solve(problem)

    # Final energy
    #exp_value = result.total_energies[0].real
    #print(f"\nTotal ground state energy = {exp_value}")

    return result

ImportError: cannot import name 'Estimator' from 'qiskit.primitives' (/home/ibmq/anaconda3/envs/qc-qiskit/lib/python3.11/site-packages/qiskit/primitives/__init__.py)

## LiH Simulation

In [ ]:
ang = 0.529177 # Bohr to Angstrom.
dist = 2.969280527*ang # Interatomic distance in angstrom.
atoms = ["Li", "H"]
coordinates = ([0.0, 0.0, 0.0], [0.0, 0.0, dist])

multiplicity = 1
charge = 0
basis = "sto-3g"

num_electrons_to_use = 4        # Should be even. LiH has originally 4 electrons.
num_spatial_orbitals_to_use = 6 # LiH has originally 6 spatial orbitals.
mapper_name = "Jordan-Wigner"   # Options: "Bravyi-Kitaev", "Jordan-Wigner", or "Parity"

problem, hamiltonian, qubit_op, mapper = get_qubit_op(
    atoms=atoms,
    coordinates=coordinates,
    multiplicity= multiplicity,
    charge = charge,
    basis = basis,
    num_electrons_to_use = num_electrons_to_use,
    num_spatial_orbitals_to_use = num_spatial_orbitals_to_use,
    mapper_name = mapper_name
)

DEBUG:qiskit_nature.second_q.drivers.pyscfd.pyscfdriver:PySCF processing messages log:
System: uname_result(system='Linux', node='lucas-Inspiron-5567', release='6.8.0-87-generic', version='#88~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Oct 14 14:03:14 UTC 2', machine='x86_64')  Threads 4
Python 3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]
numpy 1.26.0  scipy 1.16.3  h5py 3.15.1
Date: Fri Nov 21 16:32:30 2025
PySCF version 2.11.0
PySCF path  /home/lucas/anaconda3/envs/qc-qiskit/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Li     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.0000

overwrite output file: /tmp/tmp_d9vy63p.log


INFO:qiskit_nature.second_q.operators.symmetric_two_body:Attempting 4-fold to 8-fold symmetric conversion.
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Using PySCF's conversion routine
INFO:qiskit_nature.second_q.drivers.pyscfd.pyscfdriver:HF Electronic dipole moment: [-0.         0.        -4.8781301]
INFO:qiskit_nature.second_q.drivers.pyscfd.pyscfdriver:Nuclear dipole moment: [0.         0.         2.96927934]
INFO:qiskit_nature.second_q.drivers.pyscfd.pyscfdriver:Total dipole moment: [ 0.          0.         -1.90885076]
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Unfolding 8-fold to 1-fold symmetric integrals.
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Using PySCF's conversion routine
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Unfolding 8-fold to 1-fold symmetric integrals.
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Using PySCF's conversion routine
INFO:qiskit_nature.second_q.operators.symmetric_two_body:Unfolding 

Num of alpha+beta spin electrons before reduction: 4
Num of spatial orbitals before reduction: 6

Num of alpha+beta spin electrons after reduction: 4
Num of spatial orbitals after reduction: 6

Num of qubits: 12


In [ ]:
result = exact_solver(qubit_op, problem)
print(f"FCI: {result}")

DEBUG:qiskit_algorithms.eigensolvers.numpy_eigensolver:NumpyEigensolverResult:
{   'aux_operators_evaluated': None,
    'eigenstates': [   Statevector([-4.88145204e-30-1.05801365e-32j,
              5.80039427e-18+3.27141209e-18j,
             -2.69974828e-18-5.96962778e-18j, ...,
              5.05143121e-18-7.52233664e-18j,
             -1.01181441e-17-1.07469012e-17j,
             -5.49774089e-18+1.72729846e-17j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))],
    'eigenvalues': array([-8.89301591-9.29006343e-17j])}
DEBUG:qiskit_algorithms.minimum_eigensolvers.numpy_minimum_eigensolver:NumPy minimum eigensolver result: {   'aux_operators_evaluated': None,
    'eigenstate': Statevector([-4.88145204e-30-1.05801365e-32j,
              5.80039427e-18+3.27141209e-18j,
             -2.69974828e-18-5.96962778e-18j, ...,
              5.05143121e-18-7.52233664e-18j,
             -1.01181441e-17-1.07469012e-17j,
             -5.49774089e-18+1.72729846e-17j],
            dims=(2, 2,

FCI: -7.882669746647282


In [ ]:
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B
import logging

logging.basicConfig(level=logging.DEBUG)

ansatz = Ansatz(problem, mapper)
result = Adapt_VQE(ansatz,
          qubit_op,
          optimizer=SLSQP,
          gradient_threshold = 1e-5,
          eigenvalue_threshold = 1e-5,
          max_iterations = 1000,
          mapper=mapper,
          problem=problem)

DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.ucc:Gathering excitation generators...
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.ucc:Generating excitation list...
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.utils.fermionic_excitation_generator:Generated list of single alpha excitations: [(0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5)]
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.utils.fermionic_excitation_generator:Generated list of single beta excitations: [(6, 8), (6, 9), (6, 10), (6, 11), (7, 8), (7, 9), (7, 10), (7, 11)]
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.utils.fermionic_excitation_generator:Added the excitation: ((0,), (2,))
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.utils.fermionic_excitation_generator:Added the excitation: ((0,), (3,))
DEBUG:qiskit_nature.second_q.circuit.library.ansatzes.utils.fermionic_excitation_generator:Added the excitation: ((0,), (4,))
DEBUG:qiskit_nature.second_q.circuit.l

NameError: name 'Adapt_VQE' is not defined

In [ ]:
print(
    f"Number of iterations: {result.raw_result.num_iterations}"
    f"\nNumber of optimal params: {len(result.raw_result.optimal_parameters)}"
    f"\nCost function evals: {result.raw_result.cost_function_evals}"
    f"\nTotal ground state energy = {result.total_energies[0].real}"
    f"\nEigenvalues = {result.eigenvalues}"
    )

AttributeError: 'numpy.float64' object has no attribute 'raw_result'

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/book" width="45" hspace="0px" vspace="0px"></a> References<a name="ref" />

<a name="1"></a> \[1] A. Peruzzo, J. McClean, P. Shadbolt, M.-H. Yung, X.-Q. Zhou, P. J. Love, A. Aspuru-Guzik, and J. L. O’Brien, "A variational eigenvalue solver on a photonic quantum processor", [Nature Communications 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213).

<a name="2"></a> \[2] Nakanishi, K. M., Mitarai, K. & Fujii, K. Subspace-search variational quantum eigensolver for excited states. [Phys. Rev. Res. 1, 033062 (2019)](https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.1.033062).

[3] https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives

[4] https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html

[5] https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html